## Get Suspicious Requests

In [ ]:
import urllib2
import json
import os
import datetime
import csv 

# getting date from the parent path. 
path = os.getcwd().split("/") 
date = path[len(path)-1]   
dsource = path[len(path)-2]  
score_values = []

In [ ]:
try:
    import ipywidgets as widgets # For jupyter/ipython >= 1.4
except ImportError:
    from IPython.html import widgets
from IPython.display import display, HTML, clear_output, Javascript 

def fill_list(list_control,source):
    options_list = ['- Select -'] 
    options_list.extend([s for s in source])
    list_control.options = options_list

# URI panel
uri_header = widgets.HTML(value="URI", height='10%', width='100%')
uri_select = widgets.Select(height='100%', width='99%')

uri_select._css = (
    (None, 'height', '90%'),
    (None, 'width', '95%'),
    ('select', 'overflow-x', 'auto'),
    ('select', 'width', '100%'),
    ('select', 'margin', 0)
)

uri_box = widgets.Box(width='70%', height='100%')
uri_box.children = [uri_header, uri_select]

# Actions Panel
actions_header = widgets.HTML(value="&nbsp;", width='100%', height='10%')
quick_text = widgets.Text(value='', width='100%', placeholder='Quick scoring')
quick_text._css = (
    (None, 'width', '100%'),
)
rating_btn = widgets.RadioButtons(description='Rating:', options=['1', '2', '3'], width='100%')
assign_btn = widgets.Button(description='Score', width='45%')
assign_btn.button_style = 'primary'
save_btn = widgets.Button(description='Save', width='45%')
save_btn.button_style = 'primary'
save_btn._css = (
    (None, 'margin-left', '10%'),
)

actions_box = widgets.Box(width='30%', height='100%')
actions_box.children = [actions_header,quick_text,rating_btn, assign_btn,save_btn]

scoring_form = widgets.HBox(width='90%', height=250)
scoring_form.children = [uri_box, actions_box]


def data_loader(): 
    us_uris = []
    
    response = GraphQLClient.request(
        query="""query($date:SpotDateType!) {
                proxy{
                    suspicious(date:$date){
                    uri
                }
            }
        }""",
        variables={
            'date': datetime.datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d')
        }
    )
    
    if not 'errors' in response:
        for row in response['data']['proxy']['suspicious']:
            us_uris.append(row['uri'])
    else:
        print 'An error occured : '+ response['errors'][0]['message']
        
    fill_list(uri_select,us_uris)
    uri_select.value = "- Select -"   

display(Javascript("$('.widget-area > .widget-subarea > *').remove();"))
data_loader()
display(scoring_form)

# Update Suspicious Requests

In [ ]:
import csv
import datetime
import subprocess 

def assign_score(b): 
    clear_output()
    uri = quick_text.value or uri_select.value
    uri_sev = int(rating_btn.selected_label) if not "- Select -" in uri_select.value else ""
    
    clear_output()
    #Gets input values
    global score_values
    
    score_values.append((uri, uri_sev))
    
    if uri_select.value != "- Select -":
        display(Javascript("$(\"option[data-value='" + uri_select.value +"']\").remove();"))
      
    clear_output()
    data_loader()
    uri_select.value = "- Select -"
    quick_text.value = ""


def save(b):   
    variables=[]
    global score_values
    mutation="""mutation($input:[ProxyScoreInputType!]!)
                {
                  proxy{
                    score(input:$input)
                        {success}
                  }
                }""" 
    
    for row in score_values:
        variables.append({
            'date': datetime.datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d'),
            'uri': row[0] if row[0] != "" else None,
            'score': row[1] if row[1] != "" else None  
            })

    var = {'input':variables}
    response = GraphQLClient.request(mutation,var)
    
    score_values = []
    if not 'errors' in response:
        clear_output()    
        display(Javascript("$('.widget-area > .widget-subarea > *').remove();"))
        data_loader() 
        display(scoring_form)
        display(Javascript('reloadParentData();')) 
        print "Suspicious connects successfully updated"
    else:
        print "An error ocurred: " + response['errors'][0]['message']

        
assign_btn.on_click(assign_score)
save_btn.on_click(save)